In [2]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from final_code.pubmed_api import pubmed_api_pull
from final_code.language_model import check_if_text_has_outbreak
from final_code.language_model import extract_outbreak_place
import pandas as pd
import requests

In [4]:
t = ["Avian Influenza outbreak"]
n = 1

In [5]:
df = pubmed_api_pull(t, n)
df

,pmid,title,abstract,full_text,authors
0,40185841,Design and validation of a semi-quantitative m...,"Since 2001, human Metapneumovirus has been a s...",Introduction Human Metapneumovirus from discov...,"Riolo Giulia giulia.riolo@gmail.com 1, Biagini..."


In [6]:
def check_if_text_has_outbreak(text):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "gemma3:1b",
            "prompt": 
        f"""
        Analysiere den folgenden Text:

        {text}

        Frage: Enthält dieser Text Informationen über einen Krankheitsausbruch?

        Antworte bitte ausschließlich mit:
        True  --> falls es Informationen über einen Krankheitsausbruch gibt
        False --> falls es keine Informationen über einen Krankheitsausbruch gibt

        Antwort:""",
            "stream": False
        }

        )
    return response.json()["response"]

df["has_outbreak"] = df["full_text"].apply(check_if_text_has_outbreak)
df

,pmid,title,abstract,full_text,authors,has_outbreak
0,40185841,Design and validation of a semi-quantitative m...,"Since 2001, human Metapneumovirus has been a s...",Introduction Human Metapneumovirus from discov...,"Riolo Giulia giulia.riolo@gmail.com 1, Biagini...",False\n


In [ ]:
text = df.loc[0]["full_text"]
print(text)

In [23]:
answer = check_if_text_has_outbreak(text)
print(answer)

False
